In [ ]:
import os
import json
import time
import sys
import csv
import uuid
import random
import streamlit as st
import asyncio
import aiohttp
import re
import pandas as pd
import numpy as np
import together
from langchain_openai import ChatOpenAI
import google.generativeai as genai
import math
import requests
from openai import OpenAI
from dotenv import load_dotenv
from typing import List, Dict, Any, Optional, Tuple
from tqdm.asyncio import tqdm_asyncio
from concurrent.futures import ThreadPoolExecutor, as_completed
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud


c:\Users\ASUS\OneDrive\Desktop\E6data_project\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [16]:
load_dotenv()

True

In [214]:
# from datasets import load_dataset
# ds = load_dataset("Dahoas/instruct-synthetic-prompt-responses")
# train_ds = ds["train"]
# df = train_ds.to_pandas()
# df.to_csv("input.csv", index=False)

In [2]:
JUDGE_RUBRIC_PROMPT = r"""
You are an impartial expert evaluator. Your task is to evaluate an AI agent’s response
against the given prompt and ground_truth (if available).

You must return STRICTLY one JSON object following this schema:

{
  "prompt_id": "<prompt_id>",
  "scores": {
    "instruction_following": <-1 | 0 | 1>,
    "hallucination": <-1 | 0 | 1>,
    "assumption_control": <-1 | 0 | 1>,
    "coherence_accuracy": <-1 | 0 | 1>
  },
  "explanations": {
    "instruction_following": "<1 concise sentence explanation>",
    "hallucination": "<1 concise sentence explanation>",
    "assumption_control": "<1 concise sentence explanation>",
    "coherence_accuracy": "<1 concise sentence explanation>"
  },
  "total_score": <float from -1.0 to 1.0 rounded to 2 decimals>
}

--------------------
### Dimension Definitions & Scoring:

- instruction_following: Measures whether the agent follows the explicit user instructions.  
  - 1 = Perfectly followed instructions (format, count, constraints).  
  - 0 = Partially followed, minor deviation.  
  - -1 = Ignored or violated instructions.  

- hallucination: Measures factual grounding and avoidance of fabricated content.  
  - 1 = No fabricated/false claims (fully grounded in ground_truth if given).  
  - 0 = Slight speculation or unverifiable claim, but mostly correct.  
  - -1 = Contains major fabricated/false claims.  

- assumption_control: Measures whether the agent avoids or properly qualifies unjustified assumptions.  
  - 1 = No unjustified assumptions or assumptions clearly stated.  
  - 0 = Some minor assumptions not stated but not harmful.  
  - -1 = Many or major unjustified assumptions.  

- coherence_accuracy: Measures clarity, logical flow, and factual correctness.  
  - 1 = Clear, logically flowing, accurate.  
  - 0 = Somewhat clear but contains redundancy or mild confusion.  
  - -1 = Confusing, disorganized, or factually wrong.  

--------------------

### Total Score Calculation:

- Compute **total_score** as the arithmetic mean of the 4 scores above.  
  Example: (instruction_following + hallucination + assumption_control + coherence_accuracy) ÷ 4.  
- The result must be a float between **-1.0 and 1.0**, **rounded to 2 decimals**.  
- Do not invent other formulas.  
- Examples:  
  - If scores = {1, 1, 1, 1} → total_score = 1.00  
  - If scores = {1, 0, 0, 1} → total_score = 0.50  
  - If scores = {0, -1, 0, 1} → total_score = 0.00  
  - If scores = {-1, -1, -1, -1} → total_score = -1.00  

--------------------

Now evaluate the following case and return STRICT JSON only.
Prompt: '''{prompt}'''
Agent Response: '''{agent_response}'''
"""

In [17]:
# Fetch Gemini key safely
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
print(f"{GEMINI_API_KEY[:4]}")

if not GEMINI_API_KEY:
    raise ValueError("❌ GEMINI_API_KEY not found. Please set it in your environment or .env file.")

# Configure Gemini
genai.configure(api_key=GEMINI_API_KEY)

AIza


In [27]:
GROQ_API_KEY = os.getenv("GROQ_API_KEY")   
print(f"{GROQ_API_KEY[:5]}")
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")   
print(f"{OPENROUTER_API_KEY[:5]}")

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY,  # <-- replace with your key
)
model_1 = "mistralai/mistral-7b-instruct:free"
model_2 = "mistralai/mistral-small-24b-instruct-2501:free"
model_3 = "google/gemma-3-27b-it:free"
model_4 = "mistralai/mistral-small-3.1-24b-instruct:free"
model_5 = "meta-llama/llama-4-maverick:free"
model_GROQ1 = "llama-3.1-8b-instant"
model_GROQ2 = "meta-llama/llama-4-maverick-17b-128e-instruct"
model_GROQ3 = "openai/gpt-oss-120b"


gsk_2
sk-or


In [28]:

# ==== SCORING FUNCTION (Qwen Judge) ====
def score_with_qwen(prompt: str, response: str) -> dict:
    try:
        full_prompt = JUDGE_RUBRIC_PROMPT + "\n\n"
        full_prompt += f"Prompt: '''{prompt}'''\n\n"
        full_prompt += f"Agent response: '''{response}'''\n\n"
        full_prompt += "Output EXACT JSON now."

        completion = client.chat.completions.create(
            model=model_1,
            messages=[
                {"role": "system", "content": "You are an evaluator that strictly outputs JSON only."},
                {"role": "user", "content": full_prompt},
            ],
        )

        raw_text = completion.choices[0].message.content.strip()

        # Clean JSON (remove extra text around it)
        if "{" in raw_text:
            json_str = raw_text[raw_text.find("{"): raw_text.rfind("}") + 1]
        else:
            json_str = raw_text

        return json.loads(json_str)

    except Exception as e:
        return {"scores": {}, "total_score": 0, "error": str(e)}


In [29]:
def score_with_groq(prompt: str, response: str) -> dict:
    try:
        groq_key = os.environ.get("GROQ_API_KEY")
        if not groq_key:
            return {"scores": {}, "total_score": 0, "error": "Groq API key not configured"}

        url = "https://api.groq.com/openai/v1/chat/completions"
        headers = {"Authorization": f"Bearer {groq_key}"}
        
        full_prompt = JUDGE_RUBRIC_PROMPT + "\n\n"
        full_prompt += f"Prompt: '''{prompt}'''\n\n"
        full_prompt += f"Agent response: '''{response}'''\n\n"
        full_prompt += "Output EXACT JSON now."

        payload = {
            "model": model_GROQ1,   # 🔑 replace with Groq model you want
            "messages": [
                {"role": "system", "content": "You are an evaluator that strictly outputs JSON only."},
                {"role": "user", "content": full_prompt},
            ],
            "temperature": 0.0,
            "max_tokens": 512,
        }

        resp = requests.post(url, headers=headers, json=payload, timeout=60)
        resp.raise_for_status()
        data = resp.json()

        raw_text = data["choices"][0]["message"]["content"].strip()

        # Extract clean JSON
        if "{" in raw_text:
            json_str = raw_text[raw_text.find("{"): raw_text.rfind("}") + 1]
        else:
            json_str = raw_text

        return json.loads(json_str)

    except Exception as e:
        return {"scores": {}, "total_score": 0, "error": str(e)}    



In [30]:
# ==== SCORING FUNCTION ====
def score_with_gemini(prompt: str, response: str) -> dict:
    try:
        full_prompt = JUDGE_RUBRIC_PROMPT + "\n\n"
        full_prompt += f"Prompt: '''{prompt}'''\n\n"
        full_prompt += f"Agent response: '''{response}'''\n\n"
        full_prompt += "Output EXACT JSON now."

        model = genai.GenerativeModel("gemini-2.5-flash")
        result = model.generate_content(full_prompt)

        # Extract raw text
        raw_text = ""
        if result.candidates and result.candidates[0].content.parts:
            raw_text = result.candidates[0].content.parts[0].text
        else:
            raw_text = result.text

        # Clean JSON
        json_str = raw_text.strip()
        if "{" in json_str:
            json_str = json_str[json_str.find("{"): json_str.rfind("}") + 1]

        return json.loads(json_str)
    except Exception as e:
        return {"scores": {}, "total_score": 0, "error": str(e)}

In [31]:
expla =[]
def evaluate_csv(input_csv: str, output_csv: str, batch_size: int = 10, total_rows: int = 2):
    df = pd.read_csv(input_csv)
    if total_rows is None or total_rows > len(df):
        total_rows = len(df)
    print(f"📊 Total rows to process: {total_rows}")
    # If file exists, resume from last completed batch
    start_index = 0
    if os.path.exists(output_csv):
        done_df = pd.read_csv(output_csv)
        start_index = len(done_df)
        print(f"⏩ Resuming from row {start_index}")

    start_time = time.time()

    for batch_start in range(start_index, total_rows, batch_size):
        batch_end = min(batch_start + batch_size, total_rows)
        batch = df.iloc[batch_start:batch_end]

        results = []
        for i, row in batch.iterrows():
            prompt = row["prompt"]
            response = row["response"]

            judge = score_with_groq(prompt, response)  # <-- your function
            scores = judge.get("scores", {})
            # total = judge.get("total_score", 0)
            print(judge)            

            results.append({
                "prompt_id": f"Prompt_{i+1}",
                "prompt": prompt,
                "response": response,
                "instruction_following": scores.get("instruction_following", 0),
                "hallucination": scores.get("hallucination", 0),
                "assumption_control": scores.get("assumption_control", 0),
                "coherence_accuracy": scores.get("coherence_accuracy", 0),
                "total_score": np.mean([
                    scores.get("instruction_following", 0),
                    scores.get("hallucination", 0),
                    scores.get("assumption_control", 0),
                    scores.get("coherence_accuracy", 0)
                ])
            })

        # Convert batch to DataFrame
        out_df = pd.DataFrame(results)

        # Append to CSV (no header after first batch)
        if batch_start == 0 and not os.path.exists(output_csv):
            out_df.to_csv(output_csv, index=False, mode="w")
        else:
            out_df.to_csv(output_csv, index=False, mode="a", header=False)

        print(f"✅ Processed rows {batch_start+1}–{batch_end} / {total_rows}")

    elapsed = time.time() - start_time
    print(f"🎯 Finished! Total time: {elapsed/60:.2f} minutes. Output saved to {output_csv}")


In [32]:
if __name__ == "__main__":
    evaluate_csv("input.csv", "output_scored.csv")

📊 Total rows to process: 2
⏩ Resuming from row 0
{'prompt_id': '1', 'scores': {'instruction_following': 1, 'hallucination': 1, 'assumption_control': 1, 'coherence_accuracy': 1}, 'explanations': {'instruction_following': "The agent perfectly followed the user's instructions, providing a clear and detailed step-by-step guide to setting up a hydroponic garden for herbs.", 'hallucination': 'The agent provided a factual and well-grounded response, without any fabricated or false claims.', 'assumption_control': 'The agent avoided making any unjustified assumptions and clearly stated the necessary steps for setting up a hydroponic garden for herbs.', 'coherence_accuracy': "The agent's response was clear, logically flowing, and accurate, providing a well-organized and easy-to-follow guide."}, 'total_score': 1.0}
{'prompt_id': 'What type of wine goes best with steak?', 'scores': {'instruction_following': 1, 'hallucination': 1, 'assumption_control': 1, 'coherence_accuracy': 1}, 'explanations': {